En esta notebook se experimenta reconfigurando la estructura del contexto brindado, ganando en claridad y limpieza para comprobar si por esa vía el modelo es capaz de recupar la información solicitada.   

In [1]:
import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()

db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [3]:
def gpt35_system_user(system_message:str, user_message:str):
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions",
            params={
                "api-version": "2023-05-15",
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "api-key": "bd6603c266fc49389659225a451ff03d",
            },
            data=json.dumps({
                "messages": [
                    {
                        "content": system_message,
                        "role": "system"
                    },
                    {
                        "content":user_message,
                        "role": "user"
                    }
                ]
            })
        )
        choices = response.json()['choices']
        answer = {
            "status_code": response.status_code,
            "text": choices[0]['message'],
        }
        return answer

    except requests.exceptions.RequestException:
        print('HTTP Request failed')
        return None

In [4]:
contexts =  [
    [
        {
          "message": {
              "role": "user",
              "content": "Quiero un turno para mi madre"
          },
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "message_id": "1-1",
            "created_at": "20230825T12:00:00.000Z",
          },
        },
        {
          "message": {
              "role": "system",
              "content": "cual es el nombre de tu madre?"
          },
          "metadata": {
              "session_id": "1",
              "person_id": "20",
              "message_id": "1-2",
              "created_at": "20230825T12:01:00.000Z"
          }
        },
        {
            "message": {
                "role": "user",
                "content": "Alba"
            },
            "metadata": {
              "session_id": "1",
              "person_id": "20",
              "message_id": "1-3",
              "created_at": "20230825T13:00:00.000Z",
            },
        },
        {
            "message": {
                "role": "system",
                "content": "y que edad tiene?"
            },
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "message_id": "1-4",
                "created_at": "20230825T13:01:00.000Z"
            }
        },
        {
            "message": {
                "role": "user",
                "content": "66"
            },
            "metadata": {
              "session_id": "1",
            "person_id": "20",
            "message_id": "1-5",
            "created_at": "20230825T14:00:00.000Z",
          },
        },
        {
            "message": {
                "role": "system",
                "content": "perfecto, tiene turno para el 30 de agosto a las 15:00"
            },
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "message_id": "1-6",
                "created_at": "20230825T14:01:00.000Z"
            }
        }
    ],
    [
        {
            "message": {
                "role": "user",
                "content": "Cual es la capital de Colombia?"
            },
            "metadata": {
              "session_id": "4",
              "person_id": "20",
              "message_id": "4-1",
              "created_at": "20230825T15:00:00.000Z",
            },
        },
        {
            "message": {
                "role": "system",
                "content": "Bogota"
            },
            "metadata": {
                "session_id": "4",
                "person_id": "20",
                "message_id": "4-2",
                "created_at": "20230825T15:01:00.000Z"
            }
        }
    ]
  ]



In [5]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Quiero un turno para mi madre"
              },
              {
                "role": "system",
                "content": "cual es el nombre de tu madre?"
              },
              {
                "role": "user",
                "content": "Alba"
              },
              {
                "role": "system",
                "content": "y que edad tiene?"
              },
              {
                "role": "user",
                "content": "66"
              },
              {
                  "role": "system",
                  "content": "perfecto, tiene turno para el 30 de agosto a las 15:00"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20230825T12:00:00.000Z",
          },
        },
        {
            "messages": [
                {
                "role": "user",
                "content": "Cual es la capital de Colombia?"
                },
                {
                    "role": "system",
                    "content": "Bogota"
                }
            ],
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "context_id": "2",
                "created_at": "20230825T18:00:00.000Z",                
              }
        }
  ]


In [8]:
user_input_1 = "que sabes de mi madre?"
filter = {"person_id": "20"}
db_results_1 = db.similarity_search_with_score(query=user_input_1, embeddings=embeddings, filter=filter, k=1)
context_1 = "{ message: " + db_results_1[0][0].page_content + ", metadata: " + str(db_results_1[0][0].metadata) + " }"
print(f"CONTEXT:{context_1}")
system_prompt_1 = f"""
Answer the user's question this message as context.
Previous message: {context_1}
"""
answer_1 = gpt35_system_user(system_message=system_prompt_1, user_message=user_input_1)
answer_1["text"]["content"]

CONTEXT:{ message: {'role': 'user', 'content': 'Quiero un turno para mi madre'}{'role': 'system', 'content': 'cual es el nombre de tu madre?'}{'role': 'user', 'content': 'Alba'}{'role': 'system', 'content': 'y que edad tiene?'}{'role': 'user', 'content': '66'}{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}, metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20230825T12:00:00.000Z'} }


'Lo siento, como sistema de procesamiento de datos mi memoria solo retiene la información necesaria para responder a la solicitud de turnos de tu madre. No tengo acceso a información personal adicional sobre ella. ¿Hay algo más en lo que pueda ayudarte?'

El modelo vuelve a fallar con una pregunta general.

In [9]:
user_input_2 = "Una vez te pregunte por colombia no?"
filter = {"person_id": "20"}
db_results_2 = db.similarity_search_with_score(query=user_input_2, embeddings=embeddings, filter=filter, k=1)
context_2 = "{ message: " + db_results_2[0][0].page_content + ", metadata: " + str(db_results_2[0][0].metadata) + " }"
print(f"CONTEXT:{context_2}")
system_prompt_2 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.

```{context_2}```

"""
answer_2 = gpt35_system_user(system_message=system_prompt_2, user_message=user_input_2)
answer_2["text"]["content"]



CONTEXT:{ message: {'role': 'user', 'content': 'Cual es la capital de Colombia?'}{'role': 'system', 'content': 'Bogota'}, metadata: {'session_id': '1', 'person_id': '20', 'context_id': '2', 'created_at': '20230825T18:00:00.000Z'} }


'Sí, me hiciste una pregunta sobre Colombia y te respondí que la capital es Bogotá. ¿Puedo ayudarte en algo más?'

Con una pregunta específica sobre otro tema no tiene problemas.

In [10]:
user_input_3 = "Te acordas cuando era el turno de mi vieja?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.

```{context_3}```

"""
answer_3 = gpt35_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: {'role': 'user', 'content': 'Quiero un turno para mi madre'}{'role': 'system', 'content': 'cual es el nombre de tu madre?'}{'role': 'user', 'content': 'Alba'}{'role': 'system', 'content': 'y que edad tiene?'}{'role': 'user', 'content': '66'}{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}, metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20230825T12:00:00.000Z'} }


'Sí, El turno de tu madre es para el 30 de agosto a las 15:00 horas.'

Y con otra pregunta específica sobre el tema que generó la falla anteriormente vemos que se puede hacer una recuperación satisfactoria.